In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset, myNormalize, myToTensor
from torchvision import transforms

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
mynorm = myNormalize()
my_to_tensor = myToTensor()
transform = transforms.Compose([mynorm, my_to_tensor])

In [4]:
train_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\train\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)
test_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\test\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)

In [5]:
# CONFIG
NUM_WORKERS = 2
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 2

In [18]:
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [19]:
# Get image and label from custom DataLoader
img_custom, label_custom = next(iter(train_dl))

print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}")

Image shape: torch.Size([8, 12, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([8])


In [20]:
from soilnet.soil_net import SoilNetFC

In [21]:
model = SoilNetFC().to(device)
img_gpu = img_custom.to(device)
print(img_gpu.device)
y = model(img_gpu[1])
y.detach()


cuda:0


tensor([0.0501], device='cuda:0')

In [22]:
torch.min(img_gpu[0]) , torch.min(img_gpu[1])

(tensor(0., device='cuda:0'), tensor(0., device='cuda:0'))

In [23]:
def train_step(model:nn.Module, data_loader:DataLoader, loss_fn:nn.Module, optimizer:torch.optim.Optimizer):
    size = len(data_loader.dataset)
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss = 0
    
    for batch, (X, y) in enumerate(data_loader):
        # Send data to target device
        X, y = X.to(device), y.to(device)
        # 1. Forward pass
        y_pred = model(X)


        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    train_loss = train_loss / len(data_loader)
    return train_loss

In [24]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [25]:
x= train_step(model, train_dl, RMSELoss(), torch.optim.Adam(model.parameters(), lr=1e-4))
x

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.223261  [    0/   80]
loss: 0.070480  [   16/   80]
loss: 0.076602  [   32/   80]
loss: 0.051517  [   48/   80]
loss: 0.143018  [   64/   80]


0.1280114848166704

In [26]:
# Test step function
def test_step(model:nn.Module, data_loader:DataLoader, loss_fn:nn.Module):
    size = len(data_loader.dataset)
    model.eval()
    test_loss = 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(data_loader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            
            # if batch % 2 == 0:
            #     loss, current = loss.item(), batch * len(X)
            #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    test_loss /= len(data_loader)
    print(f"Test Loss: {test_loss:>8f}%")
    return test_loss

In [27]:
x= test_step(model, test_dl, RMSELoss())
x

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 0.076385%


0.07638538237661123

In [28]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = RMSELoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "test_loss": [],
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss = train_step(model=model,
                                           data_loader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss = test_step(model=model,
            data_loader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | ",
            f"train_loss: {train_loss} | ",
            f"test_loss: {test_loss} | "
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    # 6. Return the filled results at the end of the epochs
    return results

## ~~Model is to Small it can't even overfit! | try RESNET~~
## A lower learning rate (1e-5) solved out problem!

In [31]:
train(model, train_dl, test_dl, torch.optim.Adam(model.parameters(), lr=1e-5), RMSELoss(), epochs=5) # Make sure set shuffle for train back to True after testing

  0%|          | 0/5 [00:00<?, ?it/s]

loss: 0.246590  [    0/   80]
loss: 0.110782  [   16/   80]
loss: 0.208450  [   32/   80]
loss: 0.096266  [   48/   80]
loss: 0.135247  [   64/   80]


 20%|██        | 1/5 [00:07<00:30,  7.72s/it]

Test Loss: 0.122947%
Epoch: 1 |  train_loss: 0.16982422024011612 |  test_loss: 0.12294656150043011 | 
loss: 0.223298  [    0/   80]
loss: 0.091505  [   16/   80]
loss: 0.168970  [   32/   80]
loss: 0.071564  [   48/   80]
loss: 0.136772  [   64/   80]


 40%|████      | 2/5 [00:15<00:22,  7.66s/it]

...

 80%|████████  | 4/5 [00:30<00:07,  7.67s/it]

Test Loss: 0.093137%
Epoch: 4 |  train_loss: 0.13185546547174454 |  test_loss: 0.09313681609928608 | 
loss: 0.201931  [    0/   80]
loss: 0.072088  [   16/   80]
loss: 0.108124  [   32/   80]
loss: 0.045475  [   48/   80]
loss: 0.139375  [   64/   80]


100%|██████████| 5/5 [00:38<00:00,  7.66s/it]

Test Loss: 0.087456%
Epoch: 5 |  train_loss: 0.12696885578334333 |  test_loss: 0.08745554573833943 | 


{'train_loss': [0.16982422024011612,
  0.15112170279026033,
  0.13961777202785014,
  0.13185546547174454,
  0.12696885578334333],
 'test_loss': [0.12294656150043011,
  0.11022264845669269,
  0.10079749748110771,
  0.09313681609928608,
  0.08745554573833943]}